#### Speech Emotion Recognition using Librosa

In [17]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [18]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [19]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [20]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\z030590\\OneDrive - Alliance\\Desktop\\Personal\\Project\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [21]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [22]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [23]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [24]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [25]:
y_pred=model.predict(x_test)

In [26]:
y_pred

array(['happy', 'fearful', 'fearful', 'happy', 'fearful', 'fearful',
       'calm', 'disgust', 'calm', 'fearful', 'happy', 'fearful',
       'fearful', 'happy', 'disgust', 'happy', 'calm', 'fearful',
       'disgust', 'calm', 'disgust', 'disgust', 'disgust', 'calm',
       'fearful', 'happy', 'calm', 'happy', 'fearful', 'fearful', 'happy',
       'fearful', 'happy', 'fearful', 'happy', 'calm', 'calm', 'fearful',
       'calm', 'disgust', 'happy', 'calm', 'fearful', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'calm', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'happy', 'fearful', 'calm',
       'happy', 'calm', 'calm', 'disgust', 'calm', 'happy', 'happy',
       'disgust', 'calm', 'calm', 'calm', 'fearful', 'fearful', 'fearful',
       'fearful', 'fearful', 'fearful', 'fearful', 'disgust', 'fearful',
       'happy', 'fearful', 'fearful', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'fearful', 'disgust', 'fearful', 'fearful', 'fearful',
       'disgust

In [27]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 59.38%
